In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill, Alignment, Font



In [2]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
#autoscout take data
def scrape_autoscout24(driver):
    driver.get("https://www.autoscout24.com.tr/lst/bmw/i7")
    WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "ListItem_title__ndA4s"))
    )
    car_names = driver.find_elements(By.CLASS_NAME, "ListItem_title__ndA4s")
    car_prices = driver.find_elements(By.CLASS_NAME, "Price_price__APlgs")
    car_kms = driver.find_elements(By.XPATH, "//span[@data-testid='VehicleDetails-mileage_road']")
    car_locations = driver.find_elements(By.CLASS_NAME, "SellerInfo_address__leRMu")

    data = []
    for name, price, km, location in zip(car_names, car_prices, car_kms, car_locations):
        car_km_raw = km.text.strip() if km else "-"
        car_km = car_km_raw if any(char.isdigit() for char in car_km_raw) else "KM Yok"
        data.append({
            "Model": name.text.strip(),
            "Fiyat": price.text.strip(),
            "Kilometre": car_km,
            "Konum": location.text.strip() if location else "Konum Yok",
            "Kaynak": "AutoScout24"  
        })
    return data
#suchen.mobile.de take data

def scrape_mobile_de(driver):
    driver.get("https://suchen.mobile.de/fahrzeuge/search.html?dam=false&isSearchRequest=true&ms=3500%3B336%3B%3B&ref=quickSearch&s=Car&sb=rel&vc=Car")
    WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "EflMX"))
    )
    car_names = driver.find_elements(By.CLASS_NAME, "EflMX")
    car_prices = driver.find_elements(By.CLASS_NAME, "fpviJ")
    car_kms = driver.find_elements(By.CLASS_NAME, "HaBLt")
    car_locations = driver.find_elements(By.CLASS_NAME, "FPpvB")

    data = []
    for name, price, km, location in zip(car_names, car_prices, car_kms, car_locations):
        car_km_raw = km.text.strip() if km else "-"
        car_km = car_km_raw if any(char.isdigit() for char in car_km_raw) else "KM Yok"
        data.append({
            "Model": name.text.strip(),
            "Fiyat": price.text.strip(),
            "Kilometre": car_km,
            "Konum": location.text.strip() if location else "Konum Yok",
            "Kaynak": "Mobile.de"  
        })
    return data


In [ ]:
# Excel
def style_excel(file_path):
    workbook = load_workbook(file_path)
    sheet = workbook.active

    header_colors = {
        "A": {"header": "4F81BD", "content": "D9E1F2"},
        "B": {"header": "C0504D", "content": "F4CCCC"},
        "C": {"header": "9BBB59", "content": "E2EFDA"},
        "D": {"header": "8064A2", "content": "D9D2E9"},
        "E": {"header": "FF9933", "content": "FFE4B2"},  
    }
    header_font = Font(bold=True, color="FFFFFF")
    alignment = Alignment(horizontal="center", vertical="center")

    for col_letter, colors in header_colors.items():
        header_cell = sheet[f"{col_letter}1"]
        header_cell.fill = PatternFill(start_color=colors["header"], end_color=colors["header"], fill_type="solid")
        header_cell.font = header_font
        header_cell.alignment = alignment

        for row in sheet.iter_rows(min_row=2, min_col=header_cell.column, max_col=header_cell.column):
            for cell in row:
                cell.fill = PatternFill(start_color=colors["content"], end_color=colors["content"], fill_type="solid")
                cell.alignment = alignment
                if col_letter == "C" and cell.value == "KM Yok":
                    cell.fill = PatternFill(start_color="FF9999", end_color="FF9999", fill_type="solid")

    for column in sheet.columns:
        max_length = 0
        column_letter = get_column_letter(column[0].column)
        for cell in column:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        sheet.column_dimensions[column_letter].width = max_length + 2

    workbook.save(file_path)
    print(f"Excel tablosu başarıyla düzenlendi: {file_path}")

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Her iki siteden veri çek
    autoscout_data = scrape_autoscout24(driver)
    mobile_de_data = scrape_mobile_de(driver)

    combined_data = autoscout_data + mobile_de_data
    df = pd.DataFrame(combined_data)

    excel_path = "C:\\Users\\90546\\Desktop\\Cari7.xlsx"
    df.to_excel(excel_path, index=False)
    print(f"Veriler başarıyla Excel'e kaydedildi: {excel_path}")

    # Excel dosyasını düzenle
    style_excel(excel_path)

finally:
    driver.quit()

In [ ]:
##df.to_excel("C:\\Users\\90546\\Desktop\\car.xlsx",index=False)